<h1><center>Laboratorio 9:  ¿Superhéroe o Villano?  🦴</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla y Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias y Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Carolina Salgado
- Nombre de alumno 2: Sebastián Urbina


### **Link de repositorio de GitHub:** `https://github.com/SebasUrbina/MDS7202`

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [1]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
# !pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
# !pip install umap-learn

# Librería para NLP
# !pip install nltk
import nltk
# nltk.download('punkt') # sólo 1 vez
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
# nltk.download('stopwords') # sólo 1 vez

In [2]:
pd.options.display.max_columns = 30

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [3]:
# Si usted está utilizando Colaboratory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [4]:
df_comics = pd.read_csv('recursos/df_comics.csv')
df_comics_no_label = pd.read_csv('recursos/comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia

In [5]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,superpowers,alter_egos,...,has_energy_blasts,has_enhanced_senses,has_invulnerability,has_stealth,has_marksmanship,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,60,40,70,"['Super Speed', 'Super Strength']",[],...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,100,100,80,"['Accelerated Healing', 'Agility', 'Berserk Mo...",[],...,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,45,100,55,"['Energy Absorption', 'Energy Armor', 'Energy ...",[],...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,40,30,50,"['Weapon-based Powers', 'Weapons Master']",[],...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,55,55,85,"['Accelerated Healing', 'Agility', 'Astral Pro...",[],...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Se realiza un breve análisis exploratorio.

In [6]:
df_comics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1285 entries, 0 to 1366
Data columns (total 82 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Unnamed: 0                        1285 non-null   int64  
 1   name                              1285 non-null   object 
 2   real_name                         1164 non-null   object 
 3   full_name                         894 non-null    object 
 4   overall_score                     1285 non-null   object 
 5   history_text                      1285 non-null   object 
 6   powers_text                       959 non-null    object 
 7   intelligence_score                1285 non-null   int64  
 8   strength_score                    1285 non-null   int64  
 9   speed_score                       1285 non-null   int64  
 10  durability_score                  1285 non-null   int64  
 11  power_score                       1285 non-null   int64  
 12  combat_scor

In [7]:
df_comics_no_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 82 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Unnamed: 0                        84 non-null     int64  
 1   name                              82 non-null     object 
 2   real_name                         64 non-null     object 
 3   full_name                         11 non-null     object 
 4   overall_score                     84 non-null     object 
 5   history_text                      76 non-null     object 
 6   powers_text                       43 non-null     object 
 7   intelligence_score                84 non-null     int64  
 8   strength_score                    84 non-null     int64  
 9   speed_score                       84 non-null     int64  
 10  durability_score                  84 non-null     int64  
 11  power_score                       84 non-null     int64  
 12  combat_sco

## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro caso) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [8]:
docs = [
    'The teacher rocks like a good rock & roll',
    'the rock is the best actor in the world'
    ]


docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [9]:
# Definimos algunas stopword que queremos que sean eliminadas
# import nltk
# nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('english')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = [
    'The teacher rocks like a good rock & roll',
    'the rock is the best actor in the world',
    'New York is a beautiful city'
    ]

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['rock', 'best', 'actor', 'world'],
 ['new', 'york', 'beauti', 'citi']]

In [10]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [11]:
bow = CountVectorizer(tokenizer = StemmerTokenizer(), token_pattern=None)
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,actor,beauti,best,citi,good,like,new,rock,roll,teacher,world,york
0,1,0,0,0,0,1,1,0,2,1,1,0,0
1,0,1,0,1,0,0,0,0,1,0,0,1,0
2,0,0,1,0,1,0,0,1,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [12]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2), token_pattern=None)
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,& roll,actor,actor world,beauti,beauti citi,best,best actor,citi,good,good rock,like,like good,new,new york,rock,rock &,rock best,rock like,roll,teacher,teacher rock,world,york,york beauti
0,1,1,0,0,0,0,0,0,0,1,1,1,1,0,0,2,1,0,1,1,1,1,0,0,0
1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0
2,0,0,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [ ] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [ ] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

In [13]:
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']

preprocessing = ColumnTransformer(
    transformers=[
        ('text', CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2), token_pattern=None), 'history_text'),
        ('numerical', MinMaxScaler(), atributos_de_interes)
    ])

pipeline = Pipeline([
    ("preprocessor", preprocessing)
])

In [14]:
transformed_data = pipeline.fit_transform(df_comics)
transformed_data

<1285x333926 sparse matrix of type '<class 'numpy.float64'>'
	with 794030 stored elements in Compressed Sparse Row format>

In [15]:
pd.DataFrame(transformed_data.toarray(), columns=pipeline.get_feature_names_out())

,text__!,text__! !,text__! '',text__! (,text__! ),"text__! ,",text__! --,text__! .,text__! ``,text__! alison,text__! almost,text__! amazon,text__! aquaman,text__! begin,text__! boil,...,text__�kick� driven,text__�polic,text__�polic man�,text__�white,text__�white room�,text__�ǣmortal,text__�ǣmortal instruments���,text__��wors,"text__��wors ,",numerical__intelligence_score,numerical__strength_score,numerical__speed_score,numerical__durability_score,numerical__power_score,numerical__combat_score
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.85,0.3,0.60,0.60,0.40,0.70
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.80,1.0,0.80,1.00,1.00,0.80
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.80,0.5,0.55,0.45,1.00,0.55
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.80,0.1,0.25,0.40,0.30,0.50
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.90,0.4,0.45,0.55,0.55,0.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1280,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.90,0.1,0.25,0.30,1.00,0.55
1281,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.80,1.0,1.00,1.00,1.00,0.80
1282,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.95,0.5,1.00,0.75,1.00,0.80
1283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.75,0.1,1.00,0.30,1.00,0.30


## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>



Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [ ] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [ ] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

**Respuesta:**

In [14]:
pipeline_villain = Pipeline([
    ("preprocessor", preprocessing),
    ("feature_selection", SelectPercentile(score_func=f_classif, percentile=90)),
    ('clf', MultinomialNB())
])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(df_comics.drop(columns = "alignment"), 
                                                    df_comics["alignment"], 
                                                    train_size=0.8, 
                                                    test_size=0.2, 
                                                    random_state=8)

# Entrenar el modelo
pipeline_villain.fit(X_train, y_train)

# Predecir etiquetas para el conjunto de prueba
y_pred = pipeline_villain.predict(X_test)

# Calcular el classification_report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

         Bad       0.73      0.13      0.22        85
        Good       0.63      0.99      0.77       153
     Neutral       1.00      0.05      0.10        19

    accuracy                           0.63       257
   macro avg       0.79      0.39      0.36       257
weighted avg       0.69      0.63      0.54       257



```
En términos generales, el algoritmo no logra un buen desempeño en la predicción y se equivoca bastante en la clase Bad y Neutral donde tiene un recall de 0.13 y 0.05 respectivamente, lo que isgnifica que le cuesta identificar esas clases cuando realmente lo son.
```

In [22]:
from sklearn.dummy import DummyClassifier

# Comparar con un DummyClassifier estratificado
dummy = DummyClassifier(strategy='stratified', random_state=1)
dummy.fit(X_train, y_train)
y_dummy_pred = dummy.predict(X_test)

dummy_report = classification_report(y_test, y_dummy_pred)
print("Dummy Classification Report:")
print(dummy_report)

Dummy Classification Report:
              precision    recall  f1-score   support

         Bad       0.32      0.31      0.31        85
        Good       0.58      0.55      0.56       153
     Neutral       0.07      0.11      0.08        19

    accuracy                           0.44       257
   macro avg       0.32      0.32      0.32       257
weighted avg       0.45      0.44      0.44       257



```
Pese a no tener excelentes resultados con el modelo anterior, el pipeline sí es capaz de generar resultados que significan un mejor desempeño que el que tiene el modelo Dummy, lo cual es positivo pues DummyClassifier clasifica al azar las entradas.
```

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

In [15]:
pipeline_villain.get_params()

{'memory': None,
 'steps': [('preprocessor',
   ColumnTransformer(transformers=[('text',
                                    CountVectorizer(ngram_range=(1, 2),
                                                    token_pattern=None,
                                                    tokenizer=<__main__.StemmerTokenizer object at 0x000001B89C91C6D0>),
                                    'history_text'),
                                   ('numerical', MinMaxScaler(),
                                    ['intelligence_score', 'strength_score',
                                     'speed_score', 'durability_score',
                                     'power_score', 'combat_score'])])),
  ('feature_selection', SelectPercentile(percentile=90)),
  ('clf', MultinomialNB())],
 'verbose': False,
 'preprocessor': ColumnTransformer(transformers=[('text',
                                  CountVectorizer(ngram_range=(1, 2),
                                                  token_pattern=None,
  

**Respuesta:**

In [16]:
# Definir los parámetros para GridSearch
params = [
    # Clasificador 1 + hiperparámetros
    {
        'clf': [SVC(random_state=42)],
        'clf__C': [0.1,1],
        'clf__kernel': ['linear','rbf'],
        'preprocessor__text__ngram_range': [(1, 1), (1, 2), (1, 3)],
        'preprocessor__text__max_df': [0.8, 1.0], # Ignorar palabras que aparecen en más del 80% o 100% de los documentos
        'preprocessor__text__min_df': [5], # Ignorar palabras que aparecen en menos de 5 documentos
        'feature_selection__percentile': [20, 40, 60, 80],
    },
    # Clasificador 2 + hiperparámetros
    {
        'clf': [RandomForestClassifier(random_state=42)],
        'clf__max_depth': [10,None],
        'clf__n_estimators': [50,100],
        'preprocessor__text__ngram_range': [(1, 1), (1, 2), (1, 3)],
        'preprocessor__text__max_df': [0.8, 1.0], # Ignorar palabras que aparecen en más del 80% o 100% de los documentos
        'preprocessor__text__min_df': [5], # Ignorar palabras que aparecen en menos de 5 documentos
        'feature_selection__percentile': [20, 40, 60, 80],
    },
    # Clasificador 3 + hiperparámetros
    {
        'clf': [LogisticRegression(random_state=42)],
        'clf__C': [0.1,1],
        'preprocessor__text__ngram_range': [(1, 1), (1, 2), (1, 3)],
        'preprocessor__text__max_df': [0.8, 1.0] ,# Ignorar palabras que aparecen en más del 80% o 100% de los documentos
        'preprocessor__text__min_df': [5], # Ignorar palabras que aparecen en menos de 5 documentos
        'feature_selection__percentile': [20, 40, 60, 80],
    }
]

In [18]:
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV

hgs = HalvingGridSearchCV(pipeline_villain, params, n_jobs=-1, scoring='f1_macro', verbose=10, cv=3)

In [19]:
hgs.fit(X_train, y_train)

n_iterations: 4
n_required_iterations: 5
n_possible_iterations: 4
min_resources_: 18
max_resources_: 1028
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 240
n_resources: 18
Fitting 3 folds for each of 240 candidates, totalling 720 fits
----------
iter: 1
n_candidates: 80
n_resources: 54
Fitting 3 folds for each of 80 candidates, totalling 240 fits
----------
iter: 2
n_candidates: 27
n_resources: 162
Fitting 3 folds for each of 27 candidates, totalling 81 fits
----------
iter: 3
n_candidates: 9
n_resources: 486
Fitting 3 folds for each of 9 candidates, totalling 27 fits


e:\Sebas\anaconda3\envs\nlp\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


HalvingGridSearchCV(cv=3,
                    estimator=Pipeline(steps=[('preprocessor',
                                               ColumnTransformer(transformers=[('text',
                                                                                CountVectorizer(ngram_range=(1,
                                                                                                             2),
                                                                                                token_pattern=None,
                                                                                                tokenizer=<__main__.StemmerTokenizer object at 0x000001B89C91C6D0>),
                                                                                'history_text'),
                                                                               ('numerical',
                                                                                MinMaxScaler(),
                                                                                ['intelligence_score',
                                                                                 'strength_score',
                                                                                 'speed_score',
                                                                                 'durability_score',
                                                                                 'power_sc...
                                 'preprocessor__text__min_df': [5],
                                 'preprocessor__text__ngram_range': [(1, 1),
                                                                     (1, 2),
                                                                     (1, 3)]},
                                {'clf': [LogisticRegression(C=1,
                                                            random_state=42)],
                                 'clf__C': [0.1, 1],
                                 'feature_selection__percentile': [20, 40, 60,
                                                                   80],
                                 'preprocessor__text__max_df': [0.8, 1.0],
                                 'preprocessor__text__min_df': [5],
                                 'preprocessor__text__ngram_range': [(1, 1),
                                                                     (1, 2),
                                                                     (1, 3)]}],
                    scoring='f1_macro', verbose=10)

In [20]:
print(f'Best f1-macro: {hgs.best_score_}')
print(f'Best params: {hgs.best_params_}')

Best f1-macro: 0.4215892567750395
Best params: {'clf': LogisticRegression(C=1, random_state=42), 'clf__C': 1, 'feature_selection__percentile': 20, 'preprocessor__text__max_df': 1.0, 'preprocessor__text__min_df': 5, 'preprocessor__text__ngram_range': (1, 2)}


In [21]:
y_pred = hgs.predict(X_test) 
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         Bad       0.61      0.56      0.59        85
        Good       0.76      0.83      0.79       153
     Neutral       0.30      0.16      0.21        19

    accuracy                           0.69       257
   macro avg       0.55      0.52      0.53       257
weighted avg       0.67      0.69      0.68       257



Se puede observar que los resultados son mejores al modelo testeado en la parte anterior de Multinomial Naive Bayes. Pasamos de un Accuracy de 0.63 a 0.69 y una mejora relevante en el recall de la clase Bad(0.12 a 0.56) y en Neutral(0.05 a 0.16). Asimismo, los mejores resultados dan con los siguientes parámetros:

```python
{
    'clf': LogisticRegression(C=1, random_state=42), 
    'clf__C': 1, 
    'feature_selection__percentile': 20, 
    'preprocessor__text__max_df': 1.0,
    'preprocessor__text__min_df': 5, 
    'preprocessor__text__ngram_range': (1, 2)
}
```

De los modelos testeados, la regresión logística resulta ser el mejor de los modelos probados. De los parámetros obtenidos, es interesante notar el de ``feature_selection__percentile``, el cual arroja mejores resultados seleccionado el 20% de las features que poseen más importancia en función del criterio de ``f_classif`` el cual computa el test estadístico de ANOVA. En otras palabras, la selección de caracteríscas resulta arrojar los mejores resultados. Asimismo, se obtiene que usando Bigramas aporta enormemente a los resultados. Y por otro lado, ``max_df`` 1.0 nos indica que se ignoran aquellas palabras que aparecen en todos los documentos, las cuales podrían ser de manera indirecta stopwords y no estar filtradas previamente en el tokenizador que creamos. Y el parámetro de regularización ``C=1`` desempeñó mejor que 0.1.

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [53]:
df_comics_no_label = df_comics_no_label.dropna(subset=['history_text']).drop_duplicates() # eliminar ejemplos sin historia y duplicados

In [54]:
df_comics_no_label['alignment'] = hgs.predict(df_comics_no_label)

In [55]:
df_comics_no_label.loc[df_comics_no_label.name.isin(['Vergil', 'Gorilla Girl','Batcow']), ['name', 'alignment']]

,name,alignment
16,Batcow,Good
40,Gorilla Girl,Good
78,Vergil,Good


```
Finalmente podemos observar que el modelo arroja que la alineación de estos personajes es buena 😁
```

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>